<a href="https://colab.research.google.com/github/imatar/imatar-vdpac3/blob/main/storytelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importar les llibreries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [3]:
# Carregar el dataset
from google.colab import drive
drive.mount("/content/drive")

# Llegeir les dades
hotel_data = pd.read_csv(
    "/content/drive/My Drive/Visualització de dades" +
    "/hotel_bookings_clean.csv"
    )
# Mostrar les dades
hotel_data.head()

Mounted at /content/drive


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,dia,tipo
0,Resort Hotel,0.0,7.0,2015.0,July,27.0,1.0,0.0,1.0,1.0,...,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02,2015-07-01,work
1,Resort Hotel,0.0,13.0,2015.0,July,27.0,1.0,0.0,1.0,1.0,...,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02,2015-07-01,work
2,Resort Hotel,0.0,14.0,2015.0,July,27.0,1.0,0.0,2.0,2.0,...,NaN,0.0,Transient,98.0,0.0,1.0,Check-Out,2015-07-03,2015-07-01,work
3,Resort Hotel,0.0,14.0,2015.0,July,27.0,1.0,0.0,2.0,2.0,...,NaN,0.0,Transient,98.0,0.0,1.0,Check-Out,2015-07-03,2015-07-01,work
4,Resort Hotel,0.0,0.0,2015.0,July,27.0,1.0,0.0,2.0,2.0,...,NaN,0.0,Transient,107.0,0.0,0.0,Check-Out,2015-07-03,2015-07-01,work


In [ ]:
# Convertir dates al format datetime
hotel_data["reservation_status_date"] = pd.to_datetime(hotel_data["reservation_status_date"])

# Afegeir una columna amb la durada total de l'estada
hotel_data["total_nights"] = hotel_data["stays_in_weekend_nights"] + hotel_data["stays_in_week_nights"]

# Ordenar mesos
month_order = ["January", "February", "March", "April", "May", "June",
               "July", "August", "September", "October", "November", "December"]
hotel_data["arrival_date_month"] = pd.Categorical(hotel_data["arrival_date_month"], categories=month_order, ordered=True)

# Substituir valors nuls
hotel_data["children"] = hotel_data["children"].fillna(0)
hotel_data["country"] = hotel_data["country"].fillna("Unknown")

# Visualització 1

In [ ]:
# Gràfic de barres de les reserves per tipus d'hotel
total_bookings = hotel_data.groupby("hotel")["is_canceled"].count().reset_index()
total_bookings.rename(columns={"is_canceled": "total_bookings"}, inplace=True)

fig1 = px.bar(total_bookings, x='hotel', y='total_bookings', color='hotel', title="Reserves totals per tipus d'hotel",
    labels={'total_bookings': 'Nombre de reserves', 'hotel': 'Tipus d\'hotel'},
    color_discrete_map={
        'City Hotel': '#639EFA',
        'Resort Hotel': '#AB69FA' })
fig1.update_layout(showlegend=False)
fig1.show()

# Visualització 2

In [ ]:
# Gràfic de barres de les reserves anuals per tipus d'hotel
annual_bookings = hotel_data.groupby(['arrival_date_year', 'hotel'])['is_canceled'].count().reset_index()
annual_bookings.rename(columns={'is_canceled': 'total_bookings'}, inplace=True)

fig2 = px.bar(annual_bookings, x='arrival_date_year', y='total_bookings', color='hotel',
                  title="Reserves anuals diferenciades  per tipus d'hotel",
                  labels={'total_bookings': 'Nombre de reserves', 'arrival_date_year': 'Any'},
                  barmode='group',
                  color_discrete_map={
                  'City Hotel': '#639EFA',
                  'Resort Hotel': '#AB69FA' })
fig2.show()

# Visualització 3

In [ ]:
# Gràfic temporal de les reserves mensuals
hotel_data["count"] = 1
monthly_trends = hotel_data.groupby(["arrival_date_month", "hotel"])["count"].sum().reset_index()

# Crear el gràfic
fig3 = px.line(monthly_trends, x="arrival_date_month", y="count", color="hotel", title="Reserves mensuals per tipus d'hotel",
    labels={"count": "Nombre de reserves", "arrival_date_month": "Mes"}, color_discrete_map={'City Hotel': '#639EFA', 'Resort Hotel': '#AB69FA' })
fig3.update_xaxes(categoryorder="array", categoryarray=month_order)
fig3.show()

# Visualització 4

In [ ]:
# Gràfic ciruclar de les cancel·lacions
hotel_data = hotel_data.dropna(subset=['is_canceled', 'hotel'])
fig4 = make_subplots(rows=1, cols=len(hotel_data['hotel'].unique()), subplot_titles=[
    f"{hotel_type}" for hotel_type in hotel_data['hotel'].unique()],
    specs=[[{'type': 'domain'}] * len(hotel_data['hotel'].unique())])

for idx, hotel_type in enumerate(hotel_data['hotel'].unique(), start=1):
    hotel_subset = hotel_data[hotel_data['hotel'] == hotel_type]
    cancel_counts = hotel_subset['is_canceled'].value_counts(normalize=True).reset_index()
    cancel_counts.columns = ['is_canceled', 'percent']
    cancel_counts['is_canceled'] = cancel_counts['is_canceled'].map({0: 'No cancel·lat', 1: 'Cancel·lat'})

    fig4.add_trace(go.Pie(labels=cancel_counts['is_canceled'], values=cancel_counts['percent'],
                         name=hotel_type, marker=dict(colors=['forestgreen', 'firebrick'])),
                  row=1, col=idx)

fig4.update_layout(title_text="Percentatge de cancel·lacions per tipus d'hotel")
fig4.show()


# Visualització 5

In [ ]:
# Gràfic de barres de les reserves no cancel·lades
completed_reservations = hotel_data[hotel_data['is_canceled'] == 0]
completed_by_hotel = completed_reservations.groupby('hotel').size().reset_index(name='completed_reservations')

fig5 = px.bar(
    completed_by_hotel,
    x='hotel',
    y='completed_reservations',
    color='hotel',
    title="Nombre d'estades per tipus d'hotel",
    labels={'completed_reservations': 'Reserves No Cancel·lades', 'hotel': 'Tipus d\'Hotel'},
    color_discrete_map={
        'City Hotel': '#639EFA',
        'Resort Hotel': '#AB69FA' })
fig5.update_layout(showlegend=False)
fig5.show()

# Visualització 6

In [ ]:
# Gràfic de violí per la distribució de l'adr
fig6 = go.Figure()

color_map = {
    'City Hotel': '#639EFA',
    'Resort Hotel': '#AB69FA'}

for hotel_type in ['City Hotel', 'Resort Hotel']:
    fig6.add_trace(go.Violin(
        x=hotel_data['hotel'][hotel_data['hotel'] == hotel_type],
        y=hotel_data['adr'][hotel_data['hotel'] == hotel_type],
        name=hotel_type,
        meanline_visible=True,  # Mostra només la línia de la mitjana
        box_visible=True,       # Mostra el boxplot
        line_color=color_map[hotel_type]  # Assigna el color personalitzat
    ))

fig6.update_layout(
    title="Distribució dels gastos mitjans per dia per tipus d'hotel",
    xaxis_title="Tipus d'hotel",
    yaxis_title="ADR (€)",
    showlegend=False)

fig6.show()

# Visualització 7

In [ ]:
# Gràfic de violí per la durada de les estades
fig7 = go.Figure()

for hotel_type in ['City Hotel', 'Resort Hotel']:
    fig7.add_trace(go.Violin(
        x=hotel_data['hotel'][hotel_data['hotel'] == hotel_type],
        y=hotel_data['total_nights'][hotel_data['hotel'] == hotel_type],
        name=hotel_type,
        meanline_visible=True,
        box_visible=True,
         line_color=color_map[hotel_type]))

fig7.update_layout(
    title="Distribució de la durada de les estades per tipus d'hotel",
    xaxis_title="Tipus d'hotel",
    yaxis_title="Durada de l'estada (nits)",
    showlegend=False)

fig7.show()

# VIsualització 8

In [ ]:
# Gràfic circilar de la distribució del tipus de visita
visit_type_distribution = hotel_data.groupby(['hotel', 'tipo'])['is_canceled'].count().reset_index()
visit_type_distribution.rename(columns={'is_canceled': 'total_visits'}, inplace=True)

fig8 = make_subplots(
    rows=1,
    cols=len(visit_type_distribution['hotel'].unique()),  # Una columna per tipus d'hotel
    subplot_titles=[f"{hotel_type}" for hotel_type in visit_type_distribution['hotel'].unique()],
    specs=[[{'type': 'domain'}] * len(visit_type_distribution['hotel'].unique())])

for idx, hotel_type in enumerate(visit_type_distribution['hotel'].unique(), start=1):
    hotel_subset = visit_type_distribution[visit_type_distribution['hotel'] == hotel_type]
    fig8.add_trace(
        go.Pie(labels=hotel_subset['tipo'], values=hotel_subset['total_visits'],
               name=hotel_type, marker=dict(colors=px.colors.qualitative.Pastel)),
        row=1, col=idx)

fig8.update_layout(title_text="Tipus de visita segons el tipus d'hotel")
fig8.show()

# Visualització 9

In [ ]:
# Gràfic circilar de la proporció de visites amb infants
hotel_data['has_children_babies'] = (hotel_data['children'] + hotel_data['babies'] > 0)
hotel_data['has_children_babies'] = hotel_data['has_children_babies'].replace({True: 'Amb infants', False: 'Sense infants'})

children_babies_proportion = hotel_data.groupby(['hotel', 'has_children_babies'])['count'].sum().reset_index()
children_babies_proportion.rename(columns={'count': 'total_bookings'}, inplace=True)

fig9 = make_subplots(
    rows=1,
    cols=len(children_babies_proportion['hotel'].unique()),
    subplot_titles=[f"{hotel_type}" for hotel_type in children_babies_proportion['hotel'].unique()],
    specs=[[{'type': 'domain'}] * len(children_babies_proportion['hotel'].unique())])

for idx, hotel_type in enumerate(children_babies_proportion['hotel'].unique(), start=1):
    hotel_data_subset = children_babies_proportion[children_babies_proportion['hotel'] == hotel_type]
    fig9.add_trace(
        go.Pie(labels=hotel_data_subset['has_children_babies'],
               values=hotel_data_subset['total_bookings'],
               name=hotel_type,
               marker=dict(colors=px.colors.qualitative.Pastel)),
        row=1, col=idx    )

fig9.update_layout(
    title_text="Proporció de reserves amb infants i sense per tipus d'hotel")
fig9.show()

# Visualització 10

In [ ]:
# Mapa coropèctica del país d'origen
country_distribution = hotel_data.groupby("country")["count"].sum().reset_index()
country_distribution.columns = ["country", "total_bookings"]

fig10 = px.choropleth(country_distribution, locations="country", locationmode="ISO-3",
    color="total_bookings", title="Distribució de reserves per país",
    color_continuous_scale=px.colors.sequential.Plasma)

fig10.show()

# Visualització 11

In [ ]:
# Gràfic de barres apilades segons país d'origen i si la reservas s'ha cancel·lat o no
min_reserves = 1000
filtered_data = hotel_data.groupby(['country', 'hotel']).filter(lambda x: len(x) > min_reserves)

total_reserves = filtered_data.groupby(['country', 'hotel'])['count'].sum().reset_index()
total_reserves.rename(columns={'count': 'total_reserves'}, inplace=True)

cancel_reserves = filtered_data[filtered_data['is_canceled'] == 1].groupby(['country', 'hotel'])['count'].sum().reset_index()
cancel_reserves.rename(columns={'count': 'total_cancellations'}, inplace=True)

proportion_data = pd.merge(total_reserves, cancel_reserves, on=['country', 'hotel'])

fig11 = make_subplots(rows=1, cols=2,
    subplot_titles=[f"{hotel_type}" for hotel_type in proportion_data['hotel'].unique()])

for idx, hotel_type in enumerate(proportion_data['hotel'].unique(), start=1):

    hotel_subset = proportion_data[proportion_data['hotel'] == hotel_type]
    hotel_subset = hotel_subset.sort_values(by='total_reserves', ascending=False)

    fig11.add_trace(go.Bar(
        x=hotel_subset['country'],
        y=hotel_subset['total_reserves'],
        name='Reserves Totals',
        marker_color='forestgreen',
        showlegend=(idx == 1)
    ), row=1, col=idx)

    fig11.add_trace(go.Bar(
        x=hotel_subset['country'],
        y=hotel_subset['total_cancellations'],
        name='Cancel·lacions',
        marker_color='firebrick',
        showlegend=(idx == 1)
    ), row=1, col=idx)

fig11.update_layout(title_text="Distribució de reserves i cancel·lacions per país i tipus d'hotel",
    barmode='stack')
fig11.show()

# Storytelling

In [ ]:
!pip install dash
!pip install jupyter-dash

In [ ]:
fig1.update_layout(height=450, width=862.5)
fig2.update_layout(height=450, width=862.5)
fig3.update_layout(height=450, width=862.5)
fig4.update_layout(height=450, width=862.5)
fig5.update_layout(height=450, width=862.5)
fig6.update_layout(height=450, width=862.5)
fig7.update_layout(height=450, width=862.5)
fig8.update_layout(height=450, width=1725)
fig9.update_layout(height=450, width=862.5)
fig10.update_layout(height=450, width=1725)
fig11.update_layout(height=450, width=1725)

In [ ]:
from dash import Dash, dcc, html

# Inicialitza l'aplicació Dash
app = Dash(__name__)

# Layout de l'aplicació
app.layout = html.Div([
    html.H1("Anàlisi de Reserves Hoteleres", style={'textAlign': 'center'}),
    dcc.Tabs([
        # Pestanya amb fig1 i fig4
        dcc.Tab(label="Reserves totals i cancel·lacions", children=[
            html.Div([
                dcc.Graph(figure=fig1, style={'flex': '1'}),
                dcc.Graph(figure=fig4, style={'flex': '1'})
            ], style={'display': 'flex', 'justify-content': 'space-between'})
        ]),
        # Pestanya amb fig5, fig6, fig7 i fig9
        dcc.Tab(label="Dades de les estades", children=[
            html.Div([
                html.Div([
                    dcc.Graph(figure=fig5, style={'flex': '1'}),
                    dcc.Graph(figure=fig6, style={'flex': '1'})
                ], style={'display': 'flex', 'justify-content': 'space-between', 'margin-bottom': '20px'}),
                html.Div([
                    dcc.Graph(figure=fig7, style={'flex': '1'}),
                    dcc.Graph(figure=fig9, style={'flex': '1'})
                ], style={'display': 'flex', 'justify-content': 'space-between'})
            ])
        ]),
        # Pestanya amb fig8
        dcc.Tab(label="Tipus de visita", children=[
            dcc.Graph(figure=fig8, style={'width': '100%'})
         ]),
        # Pestanya amb fig2 i fig3
        dcc.Tab(label="Tendències anuals i mensuals", children=[
            html.Div([
                dcc.Graph(figure=fig2, style={'flex': '1'}),
                dcc.Graph(figure=fig3, style={'flex': '1'})
            ], style={'display': 'flex', 'justify-content': 'space-between'})
        ]),
        # Pestanya amb fig10
        dcc.Tab(label="País d'origen", children=[
            dcc.Graph(figure=fig10, style={'width': '100%'})
        ]),
        # Pestanya amb fig11
        dcc.Tab(label="Distribució de les cancel·lacions per país", children=[
                dcc.Graph(figure=fig11, style={'width': '100%'})
        ])
    ])
])

# Executa l'aplicació
if __name__ == '__main__':
    app.run_server(debug=True)